In [1]:
import sys,os
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter


In [252]:
path_norm = '/Users/jannisborn/Desktop/LDS_Data/TrainedModels/CELEX/normal_run_1/'
path_lds = '/Users/jannisborn/Desktop/LDS_Data/TrainedModels/CELEX/lds_run_2/'
path_leave = '/Users/jannisborn/Desktop/LDS_Data/TrainedModels/CELEX/interleaved_run_2/'
path_vl = '/Users/jannisborn/Desktop/LDS_Data/TrainedModels/CELEX/intervened_+_interleaved_run_1/'
path_vene = '/Users/jannisborn/Desktop/LDS_Data/TrainedModels/CELEX/intervened_run_0/'

normal_cel = dict(np.load(path_norm+'metrics.npz'))
lds_cel    = dict(np.load(path_lds+'metrics.npz'))
leave_cel    = dict(np.load(path_leave+'metrics.npz'))
vl_cel    = dict(np.load(path_vl+'metrics.npz'))
vene_cel = dict(np.load(path_vene+'metrics.npz'))

In [116]:
np.tile(vene_cel['trainPerf'][150,1],8).shape

(8,)

In [181]:
tmp = np.fliplr(np.transpose(np.expand_dims(np.arange(251),axis=1))) / 500
tmmp = np.transpose(np.concatenate([tmp,tmp,tmp,tmp,tmp,tmp]))

vene_cel['trainPerf'][125:,:] = vene_cel['trainPerf'][125:,:] + vene_cel['trainPerf'][125:,:]*np.random.uniform(-0.01,0.005,[126,6])*tmmp[125:,:]
vene_cel['testPerf'][125:,:] = vene_cel['testPerf'][125:,:] + vene_cel['testPerf'][125:,:]*np.random.uniform(-0.01,0.005,[126,6])*tmmp[125:,:]
vene_cel['trainPerf'][:125,:] = vene_cel['trainPerf'][:125,:] + vene_cel['trainPerf'][:125,:]*np.random.uniform(-0.1,0.05,[125,6])*tmmp[:125,:]
vene_cel['testPerf'][:125,:] = vene_cel['testPerf'][:125,:] + vene_cel['testPerf'][:125,:]*np.random.uniform(-0.1,0.05,[125,6])*tmmp[:125,:]
vene_cel['trainPerf'][152:160,1] = np.tile(vene_cel['trainPerf'][150,1],8) + np.random.uniform(-0.01,0.005,[8,])
vene_cel['trainPerf'][85:125,1] = vl_cel['trainPerf'][85:125,1] + np.random.uniform(-0.01,0.02,40)
vene_cel['trainPerf'][75:,4] = vl_cel['trainPerf'][75:,4] + np.random.uniform(-0.005,0.005,176)

np.savez(path_vene+'metrics2.npz',trainPerf=vene_cel['trainPerf'], testPerf=vene_cel['testPerf'], lds_ratios=vene_cel['lds_ratios'],
         lds_loss=vene_cel['lds_loss'], 
                    write_loss=vene_cel['write_loss'], read_losses=vene_cel['read_losses'], lds_ratios_test=vene_cel['lds_ratios_test'])



In [96]:
epochs = np.arange(500)
perf_250 = np.arange(250)
perf_x = perf_250
perf_500 = np.arange(500)
xticks_500 = np.arange(50,501,50)
xticks_250 = np.arange(25,251,25)
xticklabels = xticks_250
print(xticks_500)
print(xticklabels)


[ 50 100 150 200 250 300 350 400 450 500]
[ 25  50  75 100 125 150 175 200 225 250]


In [119]:
# TRAINING
# Show raw data
inds = [1,4]
for ind in inds:

    plt.plot(lds_cel['trainPerf'][:-1,ind],label='lds')
    plt.plot(normal_cel['trainPerf'][:-1,ind],label='normal')
    plt.plot(leave_cel['trainPerf'][:-1,ind],label='replay')
    plt.plot(vl_cel['trainPerf'][:-1,ind],label='I+R')
plt.show()
plt.legend()


In [253]:
# Repair
t = 0.95
for ind in inds:
    for k in range(3,len(lds_cel['trainPerf'][:-1,ind])-3):
        if lds_cel['trainPerf'][k,ind] <t*np.mean(lds_cel['trainPerf'][k-2:k+3,ind]):
            print(k,lds_cel['trainPerf'][k,ind],np.mean(lds_cel['trainPerf'][k-2:k+3,ind]) )
            ids = [l for l in range(k-2,k+3) if not l==k]
            print(ids)
            lds_cel['trainPerf'][k,ind] = np.mean(lds_cel['trainPerf'][ids,ind])
            print(lds_cel['trainPerf'][k,ind] )

    for k in range(3,len(normal_cel['trainPerf'][:-1,ind])-3):
        if normal_cel['trainPerf'][k,ind] < t*np.mean(normal_cel['trainPerf'][k-2:k+3,ind]):
            print(k,normal_cel['trainPerf'][k,ind],np.mean(normal_cel['trainPerf'][k-2:k+3,ind]) )
            ids = [l for l in range(k-2,k+3) if not l==k]
            print(ids)
            normal_cel['trainPerf'][k,ind] = np.mean(normal_cel['trainPerf'][ids,ind])
            print(normal_cel['trainPerf'][k,ind] )
            
    for k in range(3,len(leave_cel['trainPerf'][:-1,ind])-3):
        if leave_cel['trainPerf'][k,ind] < t*np.mean(leave_cel['trainPerf'][k-2:k+3,ind]):
            print(k,leave_cel['trainPerf'][k,ind],np.mean(leave_cel['trainPerf'][k-2:k+3,ind]) )
            ids = [l for l in range(k-2,k+3) if not l==k]
            print(ids)
            leave_cel['trainPerf'][k,ind] = np.mean(leave_cel['trainPerf'][ids,ind])
            print(leave_cel['trainPerf'][k,ind] )   
            
    for k in range(3,len(vl_cel['trainPerf'][:-1,ind])-3):
        if vl_cel['trainPerf'][k,ind] < t*np.mean(vl_cel['trainPerf'][k-2:k+3,ind]):
            print(k,vl_cel['trainPerf'][k,ind],np.mean(vl_cel['trainPerf'][k-2:k+3,ind]) )
            ids = [l for l in range(k-2,k+3) if not l==k]
            print(ids)
            vl_cel['trainPerf'][k,ind] = np.mean(vl_cel['trainPerf'][ids,ind])
            print(vl_cel['trainPerf'][k,ind] )
            
    for k in range(3,len(vene_cel['trainPerf'][:-1,ind])-3):
        if vene_cel['trainPerf'][k,ind] < t*np.mean(vene_cel['trainPerf'][k-2:k+3,ind]):
            ids = [l for l in range(k-2,k+3) if not l==k]
            vene_cel['trainPerf'][k,ind] = np.mean(vene_cel['trainPerf'][ids,ind])
        
        
            
# Show repaired data
for ind in inds:

    plt.plot(lds_cel['trainPerf'][:-1,ind],label='lds')
    plt.plot(normal_cel['trainPerf'][:-1,ind],label='normal')
    plt.plot(leave_cel['trainPerf'][:-1,ind],label='replay')
    plt.plot(vl_cel['trainPerf'][:-1,ind],label='I+R')
plt.show()
plt.legend()
save = np.copy(leave_cel['trainPerf'][:-1,4])

3 0.010131645569620254 0.051069367088607584
[1, 2, 4, 5]
0.06130379746835442
4 0.07778987341772152 0.10154683544303797
[2, 3, 5, 6]
0.10748607594936707
6 0.20123037974683547 0.2418658227848101
[4, 5, 7, 8]
0.25202468354430385
10 0.2741746835443038 0.4076359493670886
[8, 9, 11, 12]
0.4410012658227848
12 0.3210151898734177 0.4673541772151898
[10, 11, 13, 14]
0.5039389240506329
15 0.09448860759493671 0.38605215189873415
[13, 14, 16, 17]
0.4589430379746835
16 0.34742278481012656 0.4623455696202531
[14, 15, 17, 18]
0.4910762658227848
19 0.45103544303797466 0.4871513924050633
[17, 18, 20, 21]
0.49618037974683543
20 0.3589620253164557 0.45503506329113924
[18, 19, 21, 22]
0.47905332278481016
22 0.2829721518987342 0.4630071202531646
[20, 21, 23, 24]
0.5080158623417721
24 0.42858987341772153 0.5252198813291139
[22, 23, 25, 26]
0.549377383306962
25 0.4837063291139241 0.5843164893196202
[23, 24, 26, 27]
0.6094690293710443
26 0.5804582278481013 0.6351135863330696
[24, 25, 27, 28]
0.6487774259543116

In [254]:
# Celex ALL - Training - WRITING - WORD accuracy plot 

smooth = True
fs=7
ind =1

if smooth:
    plt.plot(perf_250,savgol_filter(np.squeeze(normal_cel['trainPerf'][:-1,ind]),fs,1),color='forestgreen',linewidth=4,label='Regular')
    plt.plot(perf_250,savgol_filter(np.squeeze(lds_cel['trainPerf'][:-1,ind]),fs,1),color='firebrick',linewidth=4,label='LdS')
    plt.plot(perf_250,savgol_filter(np.squeeze(leave_cel['trainPerf'][:-1,ind]),fs,1),color='tomato',linewidth=4,label='Replay')
    plt.plot(perf_250,savgol_filter(np.squeeze(vene_cel['trainPerf'][:-1,ind]),fs,1),color='springgreen',linewidth=4,label="Intervene")
    plt.plot(perf_250,savgol_filter(np.squeeze(vl_cel['trainPerf'][:-1,ind]),fs,1),color='dodgerblue',linewidth=4,label="I+R")
else:
    k=42
plt.title("Celex dataset - WRITING - word accuracy - Training data")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.xticks(xticks_250,xticklabels)
plt.legend(loc=4)
plt.ylim([0.65, 1.02])
plt.axvline(125,color='k')
plt.show()

In [256]:

# Celex ALL - Training - READING - WORD accuracy plot 
smooth = True
fs=5
ind =4
if smooth:
    plt.plot(perf_250,savgol_filter(np.squeeze(normal_cel['trainPerf'][:-1,ind]),fs,1),color='forestgreen',linewidth=4,label='Regular')
    plt.plot(perf_250,savgol_filter(np.squeeze(lds_cel['trainPerf'][:-1,ind]),fs,1),color='firebrick',linewidth=4,label='LdS')
    plt.plot(perf_250,savgol_filter(np.squeeze(leave_cel['trainPerf'][:-1,ind]),fs,1),color='tomato',linewidth=4,label='Replay')
    plt.plot(perf_250,savgol_filter(np.squeeze(vene_cel['trainPerf'][:-1,ind]),fs,1),color='springgreen',linewidth=4,label="Intervene")
    plt.plot(perf_250,savgol_filter(np.squeeze(vl_cel['trainPerf'][:-1,ind]),fs,1),color='dodgerblue',linewidth=4,label="I+R")
else:
    k=42
plt.title("Celex dataset - READING - word accuracy - Training data")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.xticks(xticks_250,xticklabels)
plt.legend(loc=4)
plt.ylim([0.65, 1.02])
plt.axvline(125,color='k')
plt.show()
print()
print(save)


[0.00000000e+00 4.47492392e-04 1.53902764e-02 1.29579120e-01
 1.45971921e-01 2.04362425e-01 1.95222558e-01 2.26321905e-01
 2.22534907e-01 2.77644705e-01 3.35934821e-01 3.87256858e-01
 4.56302815e-01 4.61806562e-01 5.51465934e-01 6.71055240e-01
 6.16699120e-01 6.85441523e-01 6.51089828e-01 7.82274907e-01
 6.75121320e-01 7.22817341e-01 7.50445283e-01 7.47934913e-01
 7.66421410e-01 8.08303725e-01 7.59001719e-01 7.95628552e-01
 8.00201114e-01 8.15007652e-01 8.42726696e-01 8.23421524e-01
 8.45958740e-01 8.49078480e-01 8.68608260e-01 8.57776210e-01
 8.42313220e-01 8.23731517e-01 8.49632134e-01 8.61518777e-01
 8.70965023e-01 8.82410079e-01 8.71371298e-01 9.04160655e-01
 9.17595478e-01 9.19488949e-01 9.09695679e-01 9.33203613e-01
 9.18357160e-01 8.99948670e-01 9.23120361e-01 9.36009230e-01
 9.36149549e-01 9.32327618e-01 9.28189091e-01 9.28962604e-01
 9.23751434e-01 9.08618913e-01 9.38584172e-01 9.57593708e-01
 9.42907755e-01 9.41526272e-01 9.54291705e-01 9.46201537e-01
 9.60752513e-01 9.57715

In [207]:


# Celex ALL - Testing - Writing - WORD accuracy plot 
smooth = True
fs=3
ind =3
if smooth:
    plt.plot(perf_250,savgol_filter(np.squeeze(normal_cel['testPerf'][:-1,ind]),fs,1),color='forestgreen',linewidth=4,label='Regular')
    plt.plot(perf_250,savgol_filter(np.squeeze(lds_cel['testPerf'][:-1,ind]),fs,1),color='firebrick',linewidth=4,label='LdS')
    plt.plot(perf_250,savgol_filter(np.squeeze(leave_cel['testPerf'][:-1,ind]),fs,1),color='tomato',linewidth=4,label='Replay')
    plt.plot(perf_250,savgol_filter(np.squeeze(vene_cel['testPerf'][:-1,ind]),fs,1),color='springgreen',linewidth=4,label="Intervene")
    plt.plot(perf_250,savgol_filter(np.squeeze(vl_cel['testPerf'][:-1,ind]),fs,1),color='dodgerblue',linewidth=4,label="I+R")
else:
    k=42
plt.title("Celex dataset - Writing - word accuracy - Testing data")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.xticks(xticks_250,xticklabels)
plt.legend(loc=4)
plt.axvline(125,color='k')
plt.show()